<hr style="height:0px; visibility:hidden;" />

<h1><center>5. Amplicon processing</center></h1>

---

[**Previous:** 4. Setup and QC](04-setup-QC.ipynb)
<br>

<div style="text-align: right"><a href="06-amplicon-analysis.ipynb"><b>Next:</b> 6. Amplicon analysis</a></div>

---
---

Now we're ready to start getting into the actual processing! Note that this is an R kernel we are working in, and while we will break down most of the code we see, don't feel like you need to digest and completely understand all of the R code right away.

---

## Setting up our environment

### Loading libraries

In [1]:
library(dada2)
library(tidyverse)

Loading required package: Rcpp

── Attaching packages ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.4.1     ✔ purrr   1.0.1
✔ tibble  3.1.8     ✔ dplyr   1.1.0
✔ tidyr   1.3.0     ✔ stringr 1.5.0
✔ readr   2.1.4     ✔ forcats 1.0.0
── Conflicts ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


### Setting our location and some general variables

In [2]:
# while building
setwd("~/GL4U-amplicon-tutorial/")

In [3]:
list.files()

[1] "fastqc-outputs"             "final-outputs"             
[3] "raw-reads"                  "trimmed-and-filtered-reads"

In [14]:
raw_reads_dir <- "raw-reads"
trimmed_and_filtered_reads_dir <- "trimmed-and-filtered-reads"
fastqc_outputs_dir <- "fastqc-outputs"
final_outputs_dir <- "final-outputs"

In [15]:
# reading in our sample info table
sample_info_tab <- read.table(file = "sample-info.tsv", header = TRUE, sep = "\t", row.names = 1)

<div class="alert alert-block alert-info">
<b>Code Breakdown</b>
<br>

- `read.table()`      - the primary function we're using
    - `file = `       - where we specify the input file we want to read
    - `header = `     - where we state if the first row should be treated as a header (TRUE/FALSE)
    - `sep = `        - where we specify the delimiter that separates values ("\t" is for tab)
    - `row.names = `  - where we can tell it if any columns should be treated as row names, this says to use the first column (would set to `NULL` if we wanted to explicitly not use any column as row names)

</div>

In [16]:
sample_info_tab

,treatment,color
,<chr>,<chr>
F10,flight,blue
F3,flight,blue
F5,flight,blue
F8,flight,blue
F9,flight,blue
G10,ground,chocolate4
G3,ground,chocolate4
G5,ground,chocolate4
G8,ground,chocolate4


In [18]:
sample_names <- row.names(sample_info_tab)
sample_names

[1] "F10" "F3"  "F5"  "F8"  "F9"  "G10" "G3"  "G5"  "G8"  "G9"

### Creating some variables to help with processing

In [19]:
# making an object that holds all forward read starting files
forward_raw_files <- list.files(path = raw_reads_dir, pattern = "*R1_raw.fastq.gz", full.names = TRUE)

<div class="alert alert-block alert-info">
<b>Code Breakdown</b>
<br>

- `list.files()`      - the primary function we're using
    - `path = `       - the location to look for files
    - `pattern = `    - the pattern to match, here we are using the `*` wildcard like before, to say anything that ends with "R1_raw.fastq.gz"
    - `full.names = ` - here allows us to specify if we want to retain the directory structure leading to the files, which we do want in this case (TRUE/FALSE)

</div>

In [25]:
# looking at them
forward_raw_files

[1] "raw-reads/F10_R1_raw.fastq.gz" "raw-reads/F3_R1_raw.fastq.gz" 
 [3] "raw-reads/F5_R1_raw.fastq.gz"  "raw-reads/F8_R1_raw.fastq.gz" 
 [5] "raw-reads/F9_R1_raw.fastq.gz"  "raw-reads/G10_R1_raw.fastq.gz"
 [7] "raw-reads/G3_R1_raw.fastq.gz"  "raw-reads/G5_R1_raw.fastq.gz" 
 [9] "raw-reads/G8_R1_raw.fastq.gz"  "raw-reads/G9_R1_raw.fastq.gz"

In [26]:
# now doing the same for the reverse reads
reverse_raw_files <- list.files(raw_reads_dir, pattern = "*R2_raw.fastq.gz", full.names = TRUE)
reverse_raw_files

[1] "raw-reads/F10_R2_raw.fastq.gz" "raw-reads/F3_R2_raw.fastq.gz" 
 [3] "raw-reads/F5_R2_raw.fastq.gz"  "raw-reads/F8_R2_raw.fastq.gz" 
 [5] "raw-reads/F9_R2_raw.fastq.gz"  "raw-reads/G10_R2_raw.fastq.gz"
 [7] "raw-reads/G3_R2_raw.fastq.gz"  "raw-reads/G5_R2_raw.fastq.gz" 
 [9] "raw-reads/G8_R2_raw.fastq.gz"  "raw-reads/G9_R2_raw.fastq.gz"

In [32]:
# making an object holding what will be the output trimmed/filtered forward files
forward_filtered_files <- paste0(trimmed_and_filtered_reads_dir, "/", sample_names, "_R1_filtered.fastq.gz")

<div class="alert alert-block alert-info">
<b>Code Breakdown</b>
<br>

- `paste0()`      - the primary function we're using
    - `... `      - all arguments we give as just positional arugments like the above will just be stuck together

</div>

In [33]:
forward_filtered_files

[1] "trimmed-and-filtered-reads/F10_R1_filtered.fastq.gz"
 [2] "trimmed-and-filtered-reads/F3_R1_filtered.fastq.gz" 
 [3] "trimmed-and-filtered-reads/F5_R1_filtered.fastq.gz" 
 [4] "trimmed-and-filtered-reads/F8_R1_filtered.fastq.gz" 
 [5] "trimmed-and-filtered-reads/F9_R1_filtered.fastq.gz" 
 [6] "trimmed-and-filtered-reads/G10_R1_filtered.fastq.gz"
 [7] "trimmed-and-filtered-reads/G3_R1_filtered.fastq.gz" 
 [8] "trimmed-and-filtered-reads/G5_R1_filtered.fastq.gz" 
 [9] "trimmed-and-filtered-reads/G8_R1_filtered.fastq.gz" 
[10] "trimmed-and-filtered-reads/G9_R1_filtered.fastq.gz"

In [35]:
# now doing the same for the reverse reads
reverse_filtered_files <- paste0(trimmed_and_filtered_reads_dir, "/", sample_names, "_R2_filtered.fastq.gz")
reverse_filtered_files

[1] "trimmed-and-filtered-reads/F10_R2_filtered.fastq.gz"
 [2] "trimmed-and-filtered-reads/F3_R2_filtered.fastq.gz" 
 [3] "trimmed-and-filtered-reads/F5_R2_filtered.fastq.gz" 
 [4] "trimmed-and-filtered-reads/F8_R2_filtered.fastq.gz" 
 [5] "trimmed-and-filtered-reads/F9_R2_filtered.fastq.gz" 
 [6] "trimmed-and-filtered-reads/G10_R2_filtered.fastq.gz"
 [7] "trimmed-and-filtered-reads/G3_R2_filtered.fastq.gz" 
 [8] "trimmed-and-filtered-reads/G5_R2_filtered.fastq.gz" 
 [9] "trimmed-and-filtered-reads/G8_R2_filtered.fastq.gz" 
[10] "trimmed-and-filtered-reads/G9_R2_filtered.fastq.gz"

---

## Checking on settings to remove primers

It is imperative that we properly remove the primers otherwise we will end up with non-biological sequences introduced due to the amibugous bases in the primers that were used. We can try trimming the primers with dada2's quality trimming/filtering program. But before we run it on everything, we're going to closely look at and test things on one sample – looking at the sequences before and after so we can visibly check the primers are indeed being removed.

These are the primers for this dataset, and the IUPAC degenerate-base codes.

```
f primer: GTGYCAGCMGCCGCGGTAA
r primer: GGACTACNVGGGTWTCTAAT

Y = C/T  
M = A/C  
N = A/T/G/C  
V = A/C/G  
W = A/T  
```

### Ensuring we can spot the primers

In [36]:
# establishing a connection with the F10 forward read file
fwd_test_file <- paste0(raw_reads_dir, "/F10_R1_raw.fastq.gz")

In [37]:
fwd_test_file

[1] "raw-reads/F10_R1_raw.fastq.gz"

In [38]:
incon <- gzcon(file(fwd_test_file, open = "rb"))

In [39]:
# this reads in the first 8 lines, with each set of 4 lines holding one fastq entry
fwd_lines <- readLines(incon, 8)

In [41]:
# here is how we can just get the sequences for the first 2 entries
fwd_lines[c(2,6)]

[1] "GTGCCAGCAGCCGCGGTAATACGGAGGATGCGAGCGTTATCCGGATTTAATGGGTTTAAAGGGTGCGTAGGTGGTTAATTAAGTCAGCGGTGAAAGTATGTGGCTCAACCATAAAATTGCCGTTGAAACTGGTTGACTTGAGTATAGTAGAGG"
[2] "GTGCCAGCCGCCGCGGTAATACGTAGGGGGCAAGCGTTATCCGGATTTACTGGGAGTAAAGGGAGCGTAGACGGCCAGACAAGTCTGAAGTGAAAATTCAGCGCTTAACTTTGGAAGTGCTTTGGAAACTGCCGGGATAGAGTGCAGGAGGGG"

They each start exactly with the forward primer sequence right up front, which isn't always the case (the asterisks are over the degenerate bases):

```
                           *    *
forward primer:         GTGYCAGCMGCCGCGGTAA
forward read 1 start:   GTGCCAGCAGCCGCGGTAA
forward read 2 start:   GTGCCAGCCGCCGCGGTAA
```

Let's look at a couple reverse reads:

In [42]:
# establishing a connect with the F10 forward read file
rev_test_file <- paste0(raw_reads_dir, "/F10_R2_raw.fastq.gz")

In [43]:
rev_test_file

[1] "raw-reads/F10_R2_raw.fastq.gz"

In [44]:
# establishing a connection
incon <- gzcon(file(rev_test_file, open = "rb"))

# storing the first 8 lines in a variable
rev_lines <- readLines(incon, 8)

In [45]:
# and looking at the first 2 sequences
rev_lines[c(2,6)]

[1] "GGACTACTAGGGTTTCTAATCCTGTTTGATCCCCACGCTTTCGTGCATCAGTGTCAGTTATAGAAAAGTAAGCAGCCTTCGCAAACGGAGTTCAGCGAGATATCTAAGCATTTCACCGCTANACNANGNNTTCCGCCTACCTCAAATAAACTC"
[2] "GGACTACCCGGGTTTCTAATCCTTTTTGCTCCCCACGCCTTCGAGCCTCAACGTCAGTTGCAGTCCAGCAGGACGCCTTCGCCACTGGTGTTCCTCCTAATATCTACGCATTTCACCGCTANACTANGAATTCCGCCTGCCCCTCCTGCACTC"

They each start exactly with the reverse primer sequence right up front:

```
                                **    *
reverse primer:          GGACTACNVGGGTWTCTAAT
reverse read 1 start:    GGACTACTAGGGTTTCTAAT
reverse read 2 start:    GGACTACCCGGGTTTCTAAT
```

### Doing a test trimming where we specify to cut these off
The forward primer is 19 bases, the reverse is 20. We can pass these values to the `trimLeft` argument of dada2's `filterAndTrim()` function:

In [46]:
filterAndTrim(fwd = fwd_test_file, 
              rev = rev_test_file, 
              filt = "test-F10_R1_filtered.fastq.gz",
              filt.rev = "test-F10_R2_filtered.fastq.gz", 
              trimLeft = c(19, 20))

<div class="alert alert-block alert-info">
<b>Code Breakdown</b>
<br>

- `filterAndTrim()` - primary function
    - `fwd = `      - where we provide the object holding all the forward read input file(s)
    - `rev = `      - where we provide the object holding all the reverse read input file(s)
    - `filt = `     - where we provide the object holding what will be the output forward read file(s)
    - `filt.rev = ` - where we provide the object holding what will be the output reverse read file(s)
    - `trimLeft = ` - how many bases we want to have trimmed off the left side of the reads (providing them as a vector like done above with two numbers means the first will be used for the forward reads and the second for the reverse reads)

</div>


In [47]:
list.files()

[1] "fastqc-outputs"                "final-outputs"                
[3] "raw-reads"                     "sample-info.tsv"              
[5] "test-F10_R1_filtered.fastq.gz" "test-F10_R2_filtered.fastq.gz"
[7] "trimmed-and-filtered-reads"

### Ensuring those settings successfully removed the primers
Now we are going to peek at the output trimmed files to make sure we cut off the primers, doing the same things we did above to read in part of the file and then just look at the first 2 sequences of the forward and reverse reads:

In [48]:
# establishing a connection and storing the first 8 lines into a file in one line now
fwd_filt_lines <- gzcon(file("test-F10_R1_filtered.fastq.gz", open = "rb")) %>% readLines(8)
    # reminder that this is the same as writing things nested this way
# fwd_filt_lines <- readLines(gzcon(file("F10_R1_filtered.fastq.gz", open = "rb")), 8)

In [49]:
fwd_filt_lines[c(2,6)]

[1] "TACGGAGGATGCGAGCGTTATCCGGATTTAATGGGTTTAAAGGGTGCGTAGGTGGTTAATTAAGTCAGCGGTGAAAGTATGTGGCTCAACCATAAAATTGCCGTTGAAACTGGTTGACTTGAGTATAGTAGAGG"
[2] "TACGTAGGGGGCAAGCGTTATCCGGATTTACTGGGAGTAAAGGGAGCGTAGACGGCCAGACAAGTCTGAAGTGAAAATTCAGCGCTTAACTTTGGAAGTGCTTTGGAAACTGCCGGGATAGAGTGCAGGAGGGG"

These previously started:

```
                            *    *
forward primer:          GTGYCAGCMGCCGCGGTAA
original fwd read 1:     GTGCCAGCAGCCGCGGTAA   TACGGAGGAT
original fwd read 2:     GTGCCAGCCGCCGCGGTAA   TACGTAGGGG
```

They each now begin right after the forward primer 👍

In [50]:
rev_filt_lines <- gzcon(file("test-F10_R2_filtered.fastq.gz", open = "rb")) %>% readLines(8)

In [51]:
rev_filt_lines[c(2,6)]

[1] "CCTGTTTGATCCCCACGCTTTCGTGCATCAGTGTCAGTTATAGAAAAGTAAGCAGCCTTCGCAAACGGAGTTCAGCGAGATATCTAAGCATTTCACCGCTA"
[2] "CCTTTTTGCTCCCCACGCCTTCGAGCCTCAACGTCAGTTGCAGTCCAGCAGGACGCCTTCGCCACTGGTGTTCCTCCTAATATCTACGCATTTCACCGCTA"

These previously started:

```
                                **    *
reverse primer:          GGACTACNVGGGTWTCTAAT
original rev read 1:     GGACTACTAGGGTTTCTAAT  CCTGTTTGAT
original rev read 2:     GGACTACCCGGGTTTCTAAT  CCTTTTTGCT
```

They each now begin right after the reverse primer 👍

So with that confirmation (or looking at some more samples if wanted), we can be fairly confident in using that `trimLeft` argument for all our samples to remove the primers (since these were all prepared and sequenced together the same way).

Now just removing those test output files so we know for sure we run everything the same way when we do all of them:

In [52]:
file.remove("test-F10_R1_filtered.fastq.gz", "test-F10_R2_filtered.fastq.gz")

[1] TRUE TRUE

In [53]:
list.files()

[1] "fastqc-outputs"             "final-outputs"             
[3] "raw-reads"                  "sample-info.tsv"           
[5] "trimmed-and-filtered-reads"

---

## Processing with dada2

### Quality trimming/filtering (including removing primers)

In [54]:
filtered_out <- filterAndTrim(fwd = forward_raw_files, 
                              rev = reverse_raw_files, 
                              filt = forward_filtered_files, 
                              filt.rev = reverse_filtered_files, 
                              trimLeft = c(19, 20), 
                              maxEE = c(1,1),
                              multithread = 4)

**I'm not sure which of these formats i like better yet**

<div class="alert alert-block alert-info">
<b>Code Breakdown</b>
<br>

- `filterAndTrim()` - primary function
    - `fwd = `      - where we provide the object holding all the forward read input files
    - `rev = `      - where we provide the object holding all the reverse read input files
    - `filt = `     - where we provide the object holding what will be the output forward read files
    - `filt.rev = ` - where we provide the object holding what will be the output reverse read files
    - `trimLeft = ` - how many bases we want to have trimmed off the left side of the reads (providing them as a vector like this with two numbers means the first will used for the forward reads and the second for the reverse reads)
    - `maxEE = `    - maximum "expected error" to allow for the forward and reverse reads (similar to above; you can read more about "expected error" [here](https://www.drive5.com/usearch/manual/exp_errs.html) and in its original publication [here](https://academic.oup.com/bioinformatics/article/31/21/3476/194979))

</div>


And we can check our files are where we expect:

In [56]:
list.files()

[1] "fastqc-outputs"             "final-outputs"             
[3] "raw-reads"                  "sample-info.tsv"           
[5] "trimmed-and-filtered-reads"

In [57]:
list.files(trimmed_and_filtered_reads_dir)

[1] "F10_R1_filtered.fastq.gz" "F10_R2_filtered.fastq.gz"
 [3] "F3_R1_filtered.fastq.gz"  "F3_R2_filtered.fastq.gz" 
 [5] "F5_R1_filtered.fastq.gz"  "F5_R2_filtered.fastq.gz" 
 [7] "F8_R1_filtered.fastq.gz"  "F8_R2_filtered.fastq.gz" 
 [9] "F9_R1_filtered.fastq.gz"  "F9_R2_filtered.fastq.gz" 
[11] "G10_R1_filtered.fastq.gz" "G10_R2_filtered.fastq.gz"
[13] "G3_R1_filtered.fastq.gz"  "G3_R2_filtered.fastq.gz" 
[15] "G5_R1_filtered.fastq.gz"  "G5_R2_filtered.fastq.gz" 
[17] "G8_R1_filtered.fastq.gz"  "G8_R2_filtered.fastq.gz" 
[19] "G9_R1_filtered.fastq.gz"  "G9_R2_filtered.fastq.gz"

**Now let's switch back to the [Setup and QC notebook](04-setup-QC.ipynb#Quality-assessment-of-filtered-reads) to run fastqc and multiqc on these files.**

### Generate error model of data

Now we are going to generate error models by learning the specific error-signatures of our dataset. Each sequencing run, even when all goes well, will have its own subtle variations to its error profile. dada2 tries to learn and incorporate this information when it later tries to infer the true, starting biological sequences. Here we are running the function that does this on both the forward and reverse reads.

In [60]:
# took about 45 seconds with subset dataset on local with 4 cpus
err_forward_reads <- learnErrors(fls = forward_filtered_files, multithread = 4)
err_reverse_reads <- learnErrors(fls = reverse_filtered_files, multithread = 4)

35427743 total bases in 264532 reads from 10 samples will be used for learning the error rates.
35167163 total bases in 264532 reads from 10 samples will be used for learning the error rates.


<div class="alert alert-block alert-info">
<b>Code Breakdown</b>
<br>

- `learnErrors()` - primary function
    - `fls = `          - where we provide the object holding all the input read files
    - `multithread = `  - where we can specify how many jobs to run in parallel

</div>


### Inferring sequences

Here’s where dada2 gets to do what it was born to do, that is to do its best to infer true biological sequences. It does this by incorporating the error models it generated above, quality information for the reads, and abundances of each unique sequence, and then figuring out if each sequence is more likely to be of biological origin or more likely to have been introduced by a sequencing error. You can read more about the details of this in the [dada2 paper](https://www.nature.com/articles/nmeth.3869#methods) of course or looking through their [site](https://benjjneb.github.io/dada2/index.html).

This step can be run on individual samples, which is the least computationally intensive manner, or on all samples together, which increases the function’s ability to resolve low-abundance ASVs. Imagine Sample A has 10,000 copies of sequence Z, and Sample B has 1 copy of sequence Z. Sequence Z would likely be filtered out of Sample B even though it was a “true” singleton among perhaps thousands of spurious singletons we needed to remove. Because running all samples together on large datasets can become impractical computationally, the developers also added a way to try to combine the best of both worlds that they refer to as pseudo-pooling, which is explained very nicely [on this page](https://benjjneb.github.io/dada2/pseudo.html#Pseudo-pooling). We will be using that method here:

In [62]:
# took about 20 seconds with subset dataset on local with 4 cpus
forward_seqs <- dada(derep = forward_filtered_files, err = err_forward_reads, pool = "pseudo", multithread = 4)
reverse_seqs <- dada(derep = reverse_filtered_files, err = err_reverse_reads, pool = "pseudo", multithread = 4)

Sample 1 - 26560 reads in 5943 unique sequences.
Sample 2 - 27920 reads in 5622 unique sequences.
Sample 3 - 28023 reads in 5632 unique sequences.
Sample 4 - 26796 reads in 5478 unique sequences.
Sample 5 - 24666 reads in 5149 unique sequences.
Sample 6 - 23554 reads in 4555 unique sequences.
Sample 7 - 27674 reads in 5859 unique sequences.
Sample 8 - 28505 reads in 5894 unique sequences.
Sample 9 - 28276 reads in 5248 unique sequences.
Sample 10 - 22558 reads in 4454 unique sequences.

   selfConsist step 2Sample 1 - 26560 reads in 5985 unique sequences.
Sample 2 - 27920 reads in 5528 unique sequences.
Sample 3 - 28023 reads in 5558 unique sequences.
Sample 4 - 26796 reads in 5454 unique sequences.
Sample 5 - 24666 reads in 5174 unique sequences.
Sample 6 - 23554 reads in 4375 unique sequences.
Sample 7 - 27674 reads in 5730 unique sequences.
Sample 8 - 28505 reads in 5747 unique sequences.
Sample 9 - 28276 reads in 5235 unique sequences.
Sample 10 - 22558 reads in 4381 unique sequenc

<div class="alert alert-block alert-info">
<b>Code Breakdown</b>
<br>

- `dada()` - primary function
    - `derep = ` - where we provide the object holding all the input read files
    - `err = `   - where we provide the object created by the `learnErrors()` function we ran above
    - `pool = `  - where we tell it the method to use (if any) to try to pool information across samples, as explained on [this page](https://benjjneb.github.io/dada2/pseudo.html#Pseudo-pooling)
    - `multithread = `  - where we can specify how many jobs to run in parallel

</div>


### Merging forward and reverse reads

Now dada2 merges the forward and reverse ASVs to reconstruct our full target amplicons, requiring the overlapping region to be identical between the two reads.

In [66]:
merged_amplicons <- mergePairs(dadaF = forward_seqs, derepF = forward_filtered_files, 
                               dadaR = reverse_seqs, derepR = reverse_filtered_files)

<div class="alert alert-block alert-info">
<b>Code Breakdown</b>
<br>

- `mergePairs()` - primary function
    - `dadaF = ` - where we provide the forward read object from the dada() function we ran above
    - `derepF = ` - where we provide the object holding all the input forward read files
    - `dadaR = ` - where we provide the reverse read object from the dada() function we ran above
    - `derepR = ` - where we provide the object holding all the input reverse read files

</div>


### Generating sequence table with counts per sample

Now we can generate a count table with the `makeSequenceTable()` function. This is one of the main outputs from processing an amplicon dataset. It is also often referred to as a biome table, or an OTU matrix.

In [74]:
seqtab <- makeSequenceTable(merged_amplicons)

This isn't very friendly to look at yet, because it uses the full sequences as column names, but we'll make a more traditional one where we change that in a few steps.

### Removing putative chimeras

Chimeras are technical artifacts made during PCR where different sequences merge together to form a new sequence, and this problem is extremely common during the generation of amplicon data. dada2 identifies likely chimeras by aligning each sequence with those that were recovered in greater abundance and then seeing if there are any lower-abundance sequences that can be made exactly by mixing left and right portions of two of the more-abundant ones. If so, these likely chimeric sequences are removed.

In [70]:
seqtab.nochim <- removeBimeraDenovo(unqs = seqtab, multithread = 4)

<div class="alert alert-block alert-info">
<b>Code Breakdown</b>
<br>

- `removeBimeraDenovo()` - primary function
    - `unqs = ` - where we provide the object we created with the `makeSequenceTable()` function above
    - `multithread = `  - where we can specify how many jobs to run in parallel

</div>


We can see how many unique sequences we had prior to chimera removal by looking at the number of columns in the object we made above with the `makeSequenceTable()` function:

In [79]:
ncol(seqtab)

[1] 504

And how many we had after removing likely chimeras:

In [80]:
ncol(seqtab.nochim)

[1] 85

In [81]:
ncol(seqtab.nochim) / ncol(seqtab) * 100

[1] 16.86508

That says we dropped quite a bit in terms of number of unique sequences, and we're only retaining ~17% of the total unique sequences recovered. But this is not the same as the number of actual fragments sequenced, because many of them are seen more than once. Here's a way we can look at that value:

In [82]:
sum(seqtab.nochim) / sum(seqtab) * 100

[1] 95.76756

Which tells us we retained ~96% of the initial sequences. This is very common scenario with amplicon data, having many chimeric unique sequences recovered, but only making up a small portion of the total data sequenced.

### Generating an overview of counts throughout processing

It can be helpful to have a count of how many reads we had at each step along the way in one table. This can aid in finding any potentially problematic steps. The developers’ [DADA2 tutorial](https://benjjneb.github.io/dada2/tutorial.html) provides an example of a nice, quick way to pull out how many reads were dropped at various points of the pipeline. Here’s a slightly modified version adding in a final column of percent of reads retained from the start:

In [83]:
# making a helper function
getN <- function(x) sum(getUniques(x))

summary_tab <- data.frame(row.names = sample_names,
                          starting_read_pairs = filtered_out[, 1],
                          filtered_read_pairs = filtered_out[, 2],
                          fwd_ASVs = sapply(forward_seqs, getN),
                          rev_ASVs = sapply(reverse_seqs, getN),
                          merged_ASVs = sapply(merged_amplicons, getN),
                          non_chimeras = rowSums(seqtab.nochim),
                          final_perc_reads_retained = round(rowSums(seqtab.nochim) / filtered_out[, 1] * 100, 1)
                         )

<div class="alert alert-block alert-info">

This is a very busy code block and not that straightforward for where we are at. So we aren't going to break every component down, but running each part piece-by-piece and looking at what it's doing would be good practice in the future if wanting to understand it better.
</div>


And here is what our summary table looks like:

In [84]:
summary_tab

,starting_read_pairs,filtered_read_pairs,fwd_ASVs,rev_ASVs,merged_ASVs,non_chimeras,final_perc_reads_retained
,<dbl>,<dbl>,<int>,<int>,<int>,<dbl>,<dbl>
F10,29886,26560,26532,26527,25894,24736,82.8
F3,31545,27920,27842,27870,27185,25985,82.4
F5,31726,28023,27968,28006,27455,26208,82.6
F8,30233,26796,26770,26775,26167,25174,83.3
F9,27799,24666,24654,24637,24153,22928,82.5
G10,26425,23554,23521,23524,23146,22258,84.2
G3,31392,27674,27636,27650,27034,26006,82.8
G5,32489,28505,28430,28466,27787,26726,82.3
G8,31804,28276,28206,28234,27659,26526,83.4


Showing we retained about 80% of our starting reads, with most being dropped at the initial filtering step.

### Assigning taxonomy
To assign taxonomy, we are going to use the [DECIPHER package](https://bioconductor.org/packages/release/bioc/html/DECIPHER.html). There are some DECIPHER-formatted databases available [here](http://www2.decipher.codes/Classification/TrainingSets/), which is where the one that we use below comes from.

In [85]:
# loading library used for taxonomy assignment
library(DECIPHER)

Loading required package: Biostrings

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:dplyr’:

    combine, intersect, setdiff, union


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tapply,
    union, unique, unsplit, which.max, which.min


Loading required package: S4Vectors

Loading required package: stats4


Attaching package: ‘S4Vectors’


The following objects are masked from ‘package:dplyr’:

    first, rename


The following object is masked from ‘package:tidyr’:

    expand


The 

In [86]:
# creating the type of object needed
dna <- DNAStringSet(getSequences(seqtab.nochim))
    # this is pulling the sequences out of our seqtab.nochim object, and passing them to the DNAStringSet() function

In [87]:
# downloading reference PUT THIS SOMEWHERE, MIKE, MAYBE ON THE SYSTEM, OR MAYBE JUST A BACKUP DOWNLOADABLE FROM FIGSHARE
download.file(url = "http://www2.decipher.codes/Classification/TrainingSets/SILVA_SSU_r138_2019.RData", destfile = "SILVA_SSU_r138_2019.RData")

In [88]:
# loading reference in R objects
load("SILVA_SSU_r138_2019.RData")

In [91]:
# took about 60 seconds with subset dataset on local with 4 cpus
# classifying sequences
tax_info <- IdTaxa(test = dna, trainingSet = trainingSet, strand = "both", processors = 4)


Time difference of 49.01 secs



<div class="alert alert-block alert-info">
<b>Code Breakdown</b>
<br>

- `IdTaxa()` - primary function
    - `test = ` - where we provide the dna object holding our sequences we want to classify
    - `trainingSet = ` - where we provide the object holding the reference information (it was loaded as "trainingSet" by the above `load()` function)
    - `strand = ` - specifying to check both forward and reverse strands
    - `processors = ` - where we can specify how many jobs to run in parallel

</div>


And we can peek at this object holding our classifications:

In [92]:
tax_info

  A test set of class 'Taxa' with length 85
     confidence taxon
 [1]        99% Root; Bacteria; Bacteroidota; Bacteroidia; Bacteroidales; Tan...
 [2]       100% Root; Bacteria; Firmicutes; Clostridia; Lachnospirales; Lachn...
 [3]        96% Root; Bacteria; Firmicutes; Bacilli; Erysipelotrichales; Erys...
 [4]        63% Root; Bacteria; Firmicutes; Clostridia; Oscillospirales; Rumi...
 [5]        67% Root; Bacteria; Firmicutes; Clostridia; Lachnospirales; Lachn...
 ...        ... ...
[81]        65% Root; Bacteria; Proteobacteria; Gammaproteobacteria; Oceanosp...
[82]        61% Root; Bacteria; Firmicutes; Bacilli; Paenibacillales; Paeniba...
[83]        73% Root; Bacteria; Firmicutes; Clostridia; Lachnospirales; Lachn...
[84]        74% Root; Bacteria; Bacteroidota; Bacteroidia; unclassified_Bacte...
[85]        90% Root; Bacteria; Firmicutes; Clostridia; Lachnospirales; Lachn...

In [93]:
# removing taxonomy reference file now that we no longer need it (unless i add it to the system)
unlink("SILVA_SSU_r138_2019.RData")

### Generating and writing standard outputs

The typical standard outputs form amplicon processing are: 1) a fasta file of our unique ASVs; 2) a count table showing how many times each unique ASV was detected in each sample; and 3) a taxonomy table linking our ASV IDs to their assigned taxonomy. Here is one way we can generate those files from our dada2 objects in R.

<div class="alert alert-block alert-info">

This code can get a little busy too, and it's a little beyond our to dig into it all. So like above, we aren't going to break every component down, but running each part piece-by-piece and looking at what it's doing would be good practice in the future if wanting to understand it better.
</div>


1. Making and writing out a fasta file of our recovered ASV sequences

In [100]:
# giving sequences more manageable names
asv_seqs <- colnames(seqtab.nochim)
asv_headers <- vector(dim(seqtab.nochim)[2], mode = "character")

for (i in 1:dim(seqtab.nochim)[2]) {
    asv_headers[i] <- paste(">ASV", i, sep = "_")
}

# making then writing out a fasta of final ASV sequences
asv_fasta <- c(rbind(asv_headers, asv_seqs))
write(asv_fasta, paste0(final_outputs_dir, "/ASVs.fasta"))

2. Making and writing out a count table of how many times each ASV was detected in each sample

In [101]:
# making and writing out a count table
asv_starting_tab <- t(seqtab.nochim)
colnames(asv_starting_tab) <- sample_names

asv_ids <- sub(">", "", asv_headers)

count_tab <- data.frame("ASV_ID" = asv_ids, asv_starting_tab, check.names = FALSE, row.names = NULL)

write.table(count_tab, paste0(final_outputs_dir, "/ASV_counts.tsv"), sep = "\t", quote = FALSE, row.names = FALSE)

In [97]:
# we can peek at this table with the head() function
head(count_tab)

,ASV_ID,F10,F3,F5,F8,F9,G10,G3,G5,G8,G9
,<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,ASV_1,11584,13128,12859,13238,12457,13120,11782,14679,15318,11733
2,ASV_2,1150,1539,903,887,864,789,435,1211,571,981
3,ASV_3,1017,226,220,552,515,486,3161,992,1424,693
4,ASV_4,1972,519,705,1214,1017,497,436,387,1421,790
5,ASV_5,631,1486,1653,893,728,848,794,654,211,495
6,ASV_6,560,1437,1656,777,773,895,780,604,203,520


3. Making and writing out a table of taxonomy

In [102]:
# making and writing out a table of taxonomy, with any unclassified as "NA"
ranks <- c("domain", "phylum", "class", "order", "family", "genus", "species")

starting_tax_tab <- t(sapply(tax_info, function(x) {
    m <- match(ranks, x$rank)
    taxa <- x$taxon[m]
    taxa[startsWith(taxa, "unclassified_")] <- NA
    taxa
}))

colnames(starting_tax_tab) <- ranks
tax_tab <- data.frame("ASV_ID" = asv_ids, starting_tax_tab, row.names = NULL)

write.table(tax_tab, paste0(final_outputs_dir, "/ASV_taxonomy.tsv"), sep = "\t", quote = FALSE, row.names = FALSE)

In [99]:
# we can also peek at this table
head(tax_tab)

,ASV_ID,domain,phylum,class,order,family,genus,species
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,ASV_1,Bacteria,Bacteroidota,Bacteroidia,Bacteroidales,Tannerellaceae,Parabacteroides,NA
2,ASV_2,Bacteria,Firmicutes,Clostridia,Lachnospirales,Lachnospiraceae,NA,NA
3,ASV_3,Bacteria,Firmicutes,Bacilli,Erysipelotrichales,Erysipelatoclostridiaceae,Erysipelatoclostridium,NA
4,ASV_4,Bacteria,Firmicutes,Clostridia,Oscillospirales,Ruminococcaceae,Ruminococcus,NA
5,ASV_5,Bacteria,Firmicutes,Clostridia,Lachnospirales,Lachnospiraceae,NA,NA
6,ASV_6,Bacteria,Firmicutes,Clostridia,Lachnospirales,Lachnospiraceae,NA,NA


---

And that's it for baseline processing now that we have our standard goods. **Next we'll move onto the [analysis notebook](06-amplicon-analysis.ipynb).**


---
---

[**Previous:** 4. Setup and QC](04-setup-QC.ipynb)
<br>

<div style="text-align: right"><a href="06-amplicon-analysis.ipynb"><b>Next:</b> 6. Amplicon analysis</a></div>
